In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [43]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [44]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import warnings
warnings.filterwarnings('ignore')

In [45]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [46]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('/kaggle/input/chatbot-dataset/intents.json').read() # read json file
intents = json.loads(data_file) # load json file

In [47]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)# add each elements into list
        #combination between patterns and intents
        documents.append((w, intent['tag']))#add single element into end of list
        # add to tag in our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [48]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [49]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [50]:
nltk.download('wordnet', quiet=True)

True

In [51]:
pip install --upgrade nltk

Note: you may need to restart the kernel to use updated packages.


In [ ]:
nltk.data.path.append("/root/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data")
nltk.download('wordnet', quiet=True)

In [53]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [54]:
import nltk

In [55]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents\n", documents, "\n")
# classes = intents[tag]
print (len(classes), "classes\n", classes, "\n")
# words = all words, vocabulary
print (len(words), "unique lemmatized words\n", words, "\n")
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

405 documents
 [(['Hi'], 'greeting'), (['How', 'are', 'you', '?'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['What', "'s", 'up'], 'greeting'), (['how', 'are', 'ya'], 'greeting'), (['heyy'], 'greeting'), (['whatsup'], 'greeting'), (['?', '?', '?', '?', '?', '?', '?', '?'], 'greeting'), (['cya'], 'goodbye'), (['see', 'you'], 'goodbye'), (['bye', 'bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['I', 'am', 'Leaving'], 'goodbye'), (['Bye'], 'goodbye'), (['Have', 'a', 'Good', 'day'], 'goodbye'), (['talk', 'to', 'you', 'later'], 'goodbye'), (['ttyl'], 'goodbye'), (['i', 'got', 'to', 'go'], 'goodbye'), (['gtg'], 'goodbye'), (['what', 'is', 'the', 'name', 'of', 'your', 'developers'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'your', 'creators'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'the', 'developers'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'the', 'creato

In [56]:
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words
    pattern_words = doc[0]
    # convert pattern_words in lower case
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create bag of words array,if word match found in current pattern then put 1 otherwise 0.[row * colm(263)]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # in output array 0 value for each tag ang 1 value for matched tag.[row * colm(8)]
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle training and turn into np.array
random.shuffle(training)
print("Training data created")

Training data created


In [57]:
training = np.asarray(training, dtype=object)

In [58]:
from sklearn.model_selection import train_test_split
train_x = [item[0] for item in training]
train_y = [item[1] for item in training]

In [62]:
test_size = 0.2
train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size=test_size, random_state=42)

In [ ]:
training.shape

In [59]:
train_x=np.array(train_x)
train_y=np.array(train_y)

In [63]:
test_x=np.array(test_x)
test_y=np.array(test_y)

In [ ]:
train_x.shape

In [ ]:
train_y.shape

In [61]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_x, train_y, epochs=150)

Epoch 1/150
13/13 [==============================] - 1s 3ms/step - loss: 3.6371 - accuracy: 0.0494
Epoch 2/150
13/13 [==============================] - 0s 3ms/step - loss: 3.6007 - accuracy: 0.0815
Epoch 3/150
13/13 [==============================] - 0s 2ms/step - loss: 3.5624 - accuracy: 0.0840
Epoch 4/150
13/13 [==============================] - 0s 3ms/step - loss: 3.5124 - accuracy: 0.1210
Epoch 5/150
13/13 [==============================] - 0s 3ms/step - loss: 3.4549 - accuracy: 0.1704
Epoch 6/150
13/13 [==============================] - 0s 3ms/step - loss: 3.3908 - accuracy: 0.1580
Epoch 7/150
13/13 [==============================] - 0s 3ms/step - loss: 3.2797 - accuracy: 0.1951
Epoch 8/150
13/13 [==============================] - 0s 2ms/step - loss: 3.1878 - accuracy: 0.1827
Epoch 9/150
13/13 [==============================] - 0s 3ms/step - loss: 3.0873 - accuracy: 0.2123
Epoch 10/150
13/13 [==============================] - 0s 3ms/step - loss: 2.9605 - accuracy: 0.2469
Epoch 11/

In [ ]:
y_test_predict=model.predict(x_test)

In [64]:
evaluation = model.evaluate(test_x, test_y)
print("Test Accuracy:", evaluation[1])

3/3 [==============================] - 0s 4ms/step - loss: 0.0325 - accuracy: 0.9877
Test Accuracy: 0.9876543283462524
